In [101]:
from scipy.io import arff
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np
import matplotlib.pyplot as plt

arff_file_path = './Rice_Cammeo_Osmancik.arff'

data, meta = arff.loadarff(arff_file_path)      # loading the data

df = pd.DataFrame(data)                         # put the data into data frame

df = df.sample(frac=1).reset_index(drop=True)   # shuffle the data

features = df.drop("Class", axis=1)             # extract features
label = df["Class"]                             # extract label

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.16, random_state=42) # split into training and test set

# normalize(using min-max normalization) both training and test set based on the training set max and min values
min_vals = X_train.min()
max_vals = X_train.max()
column_to_be_normalized = ['Area', 'Perimeter','Major_Axis_Length','Minor_Axis_Length','Convex_Area']

for col in X_train:
    if col in column_to_be_normalized:
        X_train[col] = (X_train[col] - min_vals[col]) / (max_vals[col] - min_vals[col])

for col in X_test:
    if col in column_to_be_normalized:
        X_test[col] = (X_test[col] - min_vals[col]) / (max_vals[col] - min_vals[col])

In [102]:
# logistic regression with gradient descent

class LogisticRegressionGD:
    def __init__(self, data, labels,learning_rate, regularized = False, reg_param = 0):
        self.weights = np.random.rand(data.shape[1])
        self.data = data
        self.labels = labels
        self.learning_rate = learning_rate
        self.N = len(data)
        self.threshold = 0.00004
        self.regularized = regularized
        self.reg_param = reg_param

    def fit(self):
        prev_weights = None

        #print(f"Lambda : {self.reg_param}")

        # apply gradient descent until no improvement on error func
        while not self.termination_condition(self.weights,prev_weights):
            prev_weights = self.weights + np.zeros(self.weights.shape[0])   # avoid pass by reference
            self.predict()
         #   print(self.calculate_error(self.weights))
            
        return self.weights

    
    def calculate_error(self,weights):
        total_sum = 0
        for i in range(self.N):
            total_sum = total_sum + np.log(1 + np.exp(-self.labels[i]*(weights @ self.data[i])))
        total_sum =  (1/self.N)*total_sum

        # regularization
        if self.regularized == True:
            total_sum += self.reg_param * (self.weights @ self.weights)

        return total_sum

    def termination_condition(self,curr_weights, prev_weights = None):
           if (prev_weights is not None) and abs(self.calculate_error(curr_weights) - self.calculate_error(prev_weights))<self.threshold:
                return True
           return False
    
    # compute gradient and take a step to the opposite direction and update weights accordingly
    def predict(self):
        gradient = 0
        for i in range(self.N):
            gradient = gradient + (self.labels[i]*self.data[i])/(1 + np.exp(self.labels[i]*(self.weights @ self.data[i])))
        gradient =  (-1/self.N)*gradient

        # derivative comes from regularization term
        if self.reg_param == True:
            gradient += 2 * self.reg_param * self.weights

        direction = -gradient
        self.weights = self.weights + self.learning_rate*direction

In [103]:
# logistic regression with stochastic gradient descent

class LogisticRegressionSGD:
    def __init__(self, data, labels,learning_rate):
        self.weights = np.random.rand(data.shape[1])
        self.data = data
        self.labels = labels
        self.learning_rate = learning_rate
        self.N = len(data)
        self.threshold = 0.00004
        self.error_data = list()

    def fit(self):
        prev_weights = None
        while True:
            random_index = None
            for round_count in range(self.N):
                random_index = np.random.randint(0, self.N)
                prev_weights = self.weights + np.zeros(self.weights.shape[0])
                self.predict(random_index)
                random_index = np.random.randint(0, self.N)
                self.error_data.append(self.calculate_error(self.weights,random_index))
            if random_index is not None and self.termination_condition(self.weights,prev_weights,random_index):
                break
                    
    def calculate_error(self,weights,random_index):
        return np.log(1 + np.exp(-self.labels[random_index]*(weights @ self.data[random_index])))

        
    def termination_condition(self,curr_weights,prev_weights,random_index):
           if (prev_weights is not None) and abs(self.calculate_error(curr_weights,random_index) - self.calculate_error(prev_weights,random_index))<self.threshold:
                return True
           return False
           
           
    def predict(self,random_index):
        direction = (self.labels[random_index]*self.data[random_index])/(1 + np.exp(self.labels[random_index]*(self.weights @ self.data[random_index])))
        self.weights = self.weights + self.learning_rate*direction

In [104]:
def evalute(test_data, test_labels, weights):
    corrects = 0
    for i in range(len(test_data)):
        s = weights @ test_data[i]
        output = 1 / (1 + np.exp(-s))

        if (output > 0.5 and test_labels[i] == 1) or (output < 0.5 and test_labels[i] == -1):
            corrects += 1
            
    acc = corrects / len(test_data)
    return acc

In [105]:
def main():
    tranformed_y_train = np.array([-1 if x == b'Cammeo' else 1 for x in y_train.values])

    kf = KFold(n_splits=5)

    lambdas = [0.1, 0.25, 0.5, 0.75, 0.9]

    lambda_accuracies = []

    for cross in range(5):
        reg_param = lambdas[cross] 
        print(f"chosen lambda : {reg_param}")
        lambda_acc = 0
        for train_index, val_index in kf.split(X_train):
            # Split the data into training and validation sets
            X_cross_train, X_val = X_train.values[train_index], X_train.values[val_index]
            y_cross_train, y_val = tranformed_y_train[train_index], tranformed_y_train[val_index]

            logistic_regression = LogisticRegressionGD(X_cross_train, y_cross_train, 0.005, regularized=True, reg_param=reg_param)
            learned_weights = logistic_regression.fit()

            lambda_acc += evalute(X_val, y_val, learned_weights) * (1 / len(lambdas))

            print(f"lambda acc : {lambda_acc}")
        print("-------------")
        lambda_accuracies.append(lambda_acc)

    print(lambdas)
    print(lambda_accuracies)

    #errors = logistic_regression.error_data
    #plt.figure(figsize=(8, 6))
    #plt.plot(range(100), errors[0:100], color='red', linestyle='-', linewidth=2, label='Straight line')
    #plt.xlabel('Iteration')
    #plt.ylabel('Error')
    #plt.title('Error Plot (for the first 100 random data points)')
    #plt.show()

if __name__ == "__main__":
    main()

chosen lambda : 0.1
lambda acc : 0.18000000000000002
lambda acc : 0.35468750000000004
lambda acc : 0.5375000000000001
lambda acc : 0.7187500000000001
lambda acc : 0.8987500000000002
-------------
chosen lambda : 0.25
lambda acc : 0.1809375
lambda acc : 0.35968750000000005
lambda acc : 0.5440625000000001
lambda acc : 0.7278125000000001
lambda acc : 0.9109375000000001
-------------
chosen lambda : 0.5
lambda acc : 0.1809375
lambda acc : 0.3615625
lambda acc : 0.54125
lambda acc : 0.725
lambda acc : 0.9059375
-------------
chosen lambda : 0.75
lambda acc : 0.180625
lambda acc : 0.36
lambda acc : 0.53625
lambda acc : 0.7153125
lambda acc : 0.89625
-------------
chosen lambda : 0.9
lambda acc : 0.180625
lambda acc : 0.36562500000000003
lambda acc : 0.5487500000000001
lambda acc : 0.7325
lambda acc : 0.9131250000000001
-------------
[0.1, 0.25, 0.5, 0.75, 0.9]
[0.8987500000000002, 0.9109375000000001, 0.9059375, 0.89625, 0.9131250000000001]
